In [1]:
import pandas as pd
import ast

# Recomendação de filmes a partir de um


Esse notebook visa comprovar a viabilidade de implementação de estratégias de recomendação baseadas em dados de um filme, para serem disponibilizadas no MovieRec.


**Dataset**: ["The Movies Dataset"](https://www.kaggle.com/rounakbanik/the-movies-dataset), disponível no Kaggle em 02/09/2020

**Atenção:** Para execução desse notebook, é necessário download externo dos arquivos do dataset, que não estão disponíveis no repositório do MovieRec. 

# Carregamento de dados

O arquivo keywords.csv contém as palavras-chave que categorizam um filme.

In [29]:
keywords = pd.read_csv(
    "keywords.csv",
    dtype={
        'id': 'UInt32',
        'keywords': 'string'
    }
)
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


O arquivo movies_metadata.csv contém atributos importantes para identificação de um filme.

In [3]:
movies = pd.read_csv(
    "movies_metadata.csv", 
    low_memory=False,
    usecols=[
        "adult",
        "belongs_to_collection",
        "budget",
        "genres",
        "id",
        "imdb_id",
        "original_language",
        "original_title",
        "overview",
        "popularity",
        "release_date",
        "runtime",
        "status",
        "tagline"
    ]
)

movies.head()

,adult,belongs_to_collection,budget,genres,id,imdb_id,original_language,original_title,overview,popularity,release_date,runtime,status,tagline
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,81.0,Released,NaN
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,104.0,Released,Roll the dice and unleash the excitement!
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,101.0,Released,Still Yelling. Still Fighting. Still Ready for...
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,127.0,Released,Friends are the people who let you be yourself...
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,1995-02-10,106.0,Released,Just When His World Is Back To Normal... He's ...


O arquivo ratings.csv contém as avaliações de usuários do TMDb, em notas de 0 a 5.

Por enquanto, carregaremos somente o arquivo ratings_small.csv, que contém apenas parte das avaliações.

In [4]:
ratings = pd.read_csv(
    "ratings_small.csv",
    dtype={
        "userId": "UInt32",
        "movieId": "UInt32",
        "rating": "float",
        "timestamp": "UInt32"
    }
)

ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


---

# Pré-processamento de dados

## Listas de palavras-chave de filmes

As palavras-chave de um filme são definidas com base em sua sinopse.

Os dados de palavras-chave de filmes estão codificados como strings que representam dicionários, usando como chave um identificador numérico. Vamos transformá-los em somente vetores com as palavras-chave. Não estamos interessados em manter os identificadores pois eles não contém valor semântico, e cada palavra-chave pode ser identificada por sua própria string.

In [30]:
keywords_l = keywords.keywords.apply(ast.literal_eval) # Inicializa o objeto dict de cada string original
keywords_l = keywords_l.apply(lambda arr : [keyword["name"] for keyword in arr]) # Constrói um array a partir do dict
keywords_l = pd.DataFrame({"id": keywords.id, "keywords": keywords_l}) # Inicializa novo DataFrame
keywords_l.head()

,id,keywords
0,862,"[jealousy, toy, boy, friendship, friends, riva..."
1,8844,"[board game, disappearance, based on children'..."
2,15602,"[fishing, best friend, duringcreditsstinger, o..."
3,31357,"[based on novel, interracial relationship, sin..."
4,11862,"[baby, midlife crisis, confidence, aging, daug..."


## Listas de gêneros de filmes

Os gêneros de cada filme estão codificados da mesma forma que as palavras-chave.

In [41]:
genres_l = movies.genres.apply(ast.literal_eval) # Inicializa o objeto dict de cada string original
genres_l = genres_l.apply(lambda arr : [genre["name"] for genre in arr]) # Constrói um array a partir do dict
genres_l = pd.DataFrame({"id": movies.id, "genres": genres_l}) # Inicializa novo DataFrame
genres_l.head()

,id,genres
0,862,"[Animation, Comedy, Family]"
1,8844,"[Adventure, Fantasy, Family]"
2,15602,"[Romance, Comedy]"
3,31357,"[Comedy, Drama, Romance]"
4,11862,[Comedy]


Embora os dados estejam agora em um formato mais familiar para processamento, eles ainda não servem de entrada para um modelo implementado pelo módulo `scikit-learn`. Para tal, precisaremos de mais transformações.

## One-Hot Encoding

Uma forma de construir o sistema de recomendações é a partir do cálculo de *vizinhos próximos* de um filme, em relação aos seus gêneros e palavras-chave. Por *proximidade*, entende-se a distância euclidiana entre filmes no espaço definido por seus *atributos*. Porém, ainda não é possível definir distâncias no espaço de palavras-chave e gêneros com o formato atual desses dados.

Utilizando a classe `MultiLabelBinarizer`, disponibilizada pelo módulo de preprocessamento `sklearn.preprocessing`, é possível codificar atributos categóricos em *One-Hot*. Nesse formato, tratamos os atributos como valores numéricos e, portanto, é possível calcular a distância entre instâncias com base em gênero e palavras-chave.

In [31]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
keywords_oh = pd.DataFrame(mlb.fit_transform(keywords_l.keywords), columns=mlb.classes_, index=keywords_l.id)
keywords_oh.head()


,'comfort women',077,10th century,1500s,15th birthday,15th century,16th century,17th century,18th century,1910s,...,부러진 화살,소원,알투비 : 리턴투베이스,오싹한 연애,위험한 소문,찌라시,찌라시 : 위험한 소문,카운트다운,하울링,형사 duelist
id,,,,,,,,,,,,,,,,,,,,,
862,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8844,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15602,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31357,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11862,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
111109,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67758,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
mlb = MultiLabelBinarizer()
genres_oh = pd.DataFrame(mlb.fit_transform(genres_l.genres), columns=mlb.classes_, index=genres_l.id)
genres_oh.head()

,Action,Adventure,Animation,Aniplex,BROSTA TV,Carousel Productions,Comedy,Crime,Documentary,Drama,...,Romance,Science Fiction,Sentai Filmworks,TV Movie,Telescene Film Group Productions,The Cartel,Thriller,Vision View Entertainment,War,Western
id,,,,,,,,,,,,,,,,,,,,,
862,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8844,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15602,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
31357,0,0,0,0,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,0,0
11862,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
